# Deep Learning with keras. NBA men's basketball ATS Prediction

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
import pandas as pd
import numpy as np 
#from bs4 import BeautifulSoup
from requests import get
import urllib.request
import csv

import matplotlib.pyplot as plt

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Import train_test_split function
from sklearn.model_selection import train_test_split

# Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

from sklearn import tree
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import AdaBoostRegressor
from sklearn import tree
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import random

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'figure.figsize':(12,8), 'figure.dpi':100})

In [2]:
seed = np.random.seed(777) #for reproducibility

In [3]:
nba = pd.read_csv('GAMES_LAST10_YEARS.csv')
nba.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,HOME_TEAM_ABB,AWAY_TEAM_ABB,HOME_TEAM_NAME,HOME_TEAM_W_TODATE,AWAY_TEAM_NAME,AWAY_TEAM_W_TODATE
0,4/17/2021,22000856,Final,1610612764,1610612765,2020,1610612764,121,0.532,0.632,...,0.333,19,45,1,WAS,DET,Washington,23,Detroit,17
1,3/28/2021,22000706,Final,1610612747,1610612753,2020,1610612747,96,0.415,0.739,...,0.333,19,51,1,LAL,ORL,L.A. Lakers,30,Orlando,15
2,3/16/2021,22000611,Final,1610612755,1610612752,2020,1610612755,99,0.411,0.737,...,0.333,19,47,1,PHI,NYK,Philadelphia,28,New York,20
3,2/12/2020,21900816,Final,1610612762,1610612748,2019,1610612762,116,0.482,0.870,...,0.333,19,37,1,UTA,MIA,Utah,36,Miami,35
4,1/26/2020,21900685,Final,1610612743,1610612745,2019,1610612743,117,0.473,0.900,...,0.333,19,44,1,DEN,HOU,Denver,32,Houston,28


In [4]:
nba.columns

Index(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID',
       'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home', 'FG_PCT_home',
       'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'TEAM_ID_away',
       'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away', 'HOME_TEAM_WINS', 'HOME_TEAM_ABB', 'AWAY_TEAM_ABB',
       'HOME_TEAM_NAME', 'HOME_TEAM_W_TODATE', 'AWAY_TEAM_NAME',
       'AWAY_TEAM_W_TODATE'],
      dtype='object')

In [5]:
nba = nba.sample(frac=1).reset_index(drop=True)

In [6]:
nba.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,HOME_TEAM_ABB,AWAY_TEAM_ABB,HOME_TEAM_NAME,HOME_TEAM_W_TODATE,AWAY_TEAM_NAME,AWAY_TEAM_W_TODATE
0,4/3/2018,21701158,Final,1610612748,1610612737,2017,1610612748,101,0.451,0.846,...,0.394,24,49,1,MIA,ATL,Miami,42,Atlanta,22
1,11/1/2017,21700109,Final,1610612738,1610612758,2017,1610612738,113,0.488,0.842,...,0.300,24,26,1,BOS,SAC,Boston,6,Sacramento,1
2,2/2/2022,22100773,Final,1610612754,1610612753,2021,1610612754,118,0.449,0.696,...,0.314,29,52,0,IND,ORL,Indiana,19,Orlando,12
3,2/18/2016,21500803,Final,1610612739,1610612741,2015,1610612739,106,0.462,0.882,...,0.381,12,49,1,CLE,CHI,Cleveland,39,Chicago,27
4,11/5/2015,21500073,Final,1610612743,1610612762,2015,1610612743,84,0.389,0.676,...,0.458,23,43,0,DEN,UTA,Denver,2,Utah,3


In [7]:
nba.describe()

,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,...,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,HOME_TEAM_W_TODATE,AWAY_TEAM_W_TODATE
count,1.445000e+04,1.445000e+04,1.445000e+04,14450.000000,1.445000e+04,14450.000000,14450.000000,14450.000000,14450.000000,14450.000000,...,1.445000e+04,14450.000000,14450.000000,14450.000000,14450.000000,14450.000000,14450.000000,14450.000000,14450.000000,14450.000000
mean,2.216603e+07,1.610613e+09,1.610613e+09,2015.931765,1.610613e+09,105.872042,0.460638,0.763124,0.357558,23.414879,...,1.610613e+09,103.419239,0.450786,0.761578,0.349824,22.333218,43.110450,0.576955,20.516747,20.431972
std,5.407421e+06,8.638377e+00,8.653594e+00,3.037368,8.638377e+00,13.092912,0.055740,0.102643,0.098413,5.139091,...,8.653594e+00,13.324255,0.054698,0.105251,0.097496,5.111793,6.524356,0.494060,15.416573,15.324816
min,1.120000e+07,1.610613e+09,1.610613e+09,2011.000000,1.610613e+09,59.000000,0.257000,0.143000,0.000000,8.000000,...,1.610613e+09,56.000000,0.246000,0.143000,0.000000,5.000000,20.000000,0.000000,0.000000,0.000000
25%,2.130058e+07,1.610613e+09,1.610613e+09,2013.000000,1.610613e+09,97.000000,0.422000,0.700000,0.294000,20.000000,...,1.610613e+09,94.000000,0.414000,0.696000,0.286000,19.000000,39.000000,0.000000,8.000000,7.000000
50%,2.160051e+07,1.610613e+09,1.610613e+09,2016.000000,1.610613e+09,105.000000,0.459000,0.769000,0.357000,23.000000,...,1.610613e+09,103.000000,0.450000,0.767000,0.350000,22.000000,43.000000,1.000000,18.000000,18.000000
75%,2.190043e+07,1.610613e+09,1.610613e+09,2018.000000,1.610613e+09,115.000000,0.500000,0.833000,0.421000,27.000000,...,1.610613e+09,112.000000,0.488000,0.833000,0.414000,26.000000,47.000000,1.000000,31.000000,31.000000
max,5.200021e+07,1.610613e+09,1.610613e+09,2021.000000,1.610613e+09,161.000000,0.684000,1.000000,0.842000,50.000000,...,1.610613e+09,168.000000,0.687000,1.000000,0.783000,46.000000,81.000000,1.000000,73.000000,73.000000


In [8]:
VARS = nba[['PTS_home', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home','PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away', 'HOME_TEAM_W_TODATE','AWAY_TEAM_W_TODATE']]

In [9]:
VARS.head()

,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_W_TODATE,AWAY_TEAM_W_TODATE
0,101,0.451,0.846,0.242,22,46,98,0.420,0.733,0.394,24,49,42,22
1,113,0.488,0.842,0.415,30,52,86,0.432,0.833,0.300,24,26,6,1
2,118,0.449,0.696,0.438,25,46,119,0.442,0.923,0.314,29,52,19,12
3,106,0.462,0.882,0.292,21,52,95,0.398,0.591,0.381,12,49,39,27
4,84,0.389,0.676,0.250,16,45,96,0.435,0.647,0.458,23,43,2,3


In [10]:
VARS.shape

(14450, 14)

In [11]:
VARS.isna().sum()

PTS_home              0
FG_PCT_home           0
FT_PCT_home           0
FG3_PCT_home          0
AST_home              0
REB_home              0
PTS_away              0
FG_PCT_away           0
FT_PCT_away           0
FG3_PCT_away          0
AST_away              0
REB_away              0
HOME_TEAM_W_TODATE    0
AWAY_TEAM_W_TODATE    0
dtype: int64

In [12]:
#import seaborn as sns
#corr = away_spread_cov.corr()
#sns.heatmap(corr,xticklabels=corr.columns.values, yticklabels=corr.columns.values)


In [13]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({'figure.figsize':(12,8), 'figure.dpi':300})

corr = VARS.corr()
corr.style.background_gradient(cmap='gray').set_precision(2)

C:\Users\dudad\AppData\Local\Temp\ipykernel_14408\3906621680.py:6: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  corr.style.background_gradient(cmap='gray').set_precision(2)


,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_W_TODATE,AWAY_TEAM_W_TODATE
PTS_home,1.00,0.68,0.17,0.46,0.60,0.15,0.44,0.11,0.06,0.05,0.19,-0.20,0.12,-0.02
FG_PCT_home,0.68,1.00,0.02,0.50,0.55,-0.18,0.10,0.03,0.02,0.01,0.02,-0.50,0.13,-0.01
FT_PCT_home,0.17,0.02,1.00,0.03,0.01,-0.06,0.06,0.04,0.03,0.02,0.02,-0.13,0.05,0.03
FG3_PCT_home,0.46,0.50,0.03,1.00,0.38,-0.10,0.04,0.01,0.01,-0.01,-0.00,-0.25,0.07,-0.02
AST_home,0.60,0.55,0.01,0.38,1.00,0.06,0.17,0.01,0.02,-0.01,0.15,-0.14,0.08,-0.04
REB_home,0.15,-0.18,-0.06,-0.10,0.06,1.00,-0.20,-0.50,-0.12,-0.26,-0.14,0.04,0.01,-0.06
PTS_away,0.44,0.10,0.06,0.04,0.17,-0.20,1.00,0.68,0.20,0.46,0.60,0.15,-0.05,0.08
FG_PCT_away,0.11,0.03,0.04,0.01,0.01,-0.50,0.68,1.00,0.04,0.49,0.54,-0.17,-0.03,0.10
FT_PCT_away,0.06,0.02,0.03,0.01,0.02,-0.12,0.20,0.04,1.00,0.03,0.03,-0.04,0.00,0.04
FG3_PCT_away,0.05,0.01,0.02,-0.01,-0.01,-0.26,0.46,0.49,0.03,1.00,0.37,-0.10,-0.01,0.05


In [14]:
df1 = VARS
df1.shape

(14450, 14)

In [15]:
df1.head()

,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_W_TODATE,AWAY_TEAM_W_TODATE
0,101,0.451,0.846,0.242,22,46,98,0.420,0.733,0.394,24,49,42,22
1,113,0.488,0.842,0.415,30,52,86,0.432,0.833,0.300,24,26,6,1
2,118,0.449,0.696,0.438,25,46,119,0.442,0.923,0.314,29,52,19,12
3,106,0.462,0.882,0.292,21,52,95,0.398,0.591,0.381,12,49,39,27
4,84,0.389,0.676,0.250,16,45,96,0.435,0.647,0.458,23,43,2,3


In [16]:
y1=df1[['PTS_home']]
y2=df1[['PTS_away']]

In [17]:
X = df1.drop(['PTS_away', 'PTS_home','FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away','REB_away','AWAY_TEAM_W_TODATE'],axis=1)
X1 = df1.drop(['PTS_home', 'PTS_away', 'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'HOME_TEAM_W_TODATE'],axis=1)

In [18]:
#from sklearn.preprocessing import LabelEncoder
# encode class values as integers
#encoder = LabelEncoder()
#encoder.fit(y)
#encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
#dummy_y = np_utils.to_categorical(encoded_Y)


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y1, test_size = 0.33, random_state = seed)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y2, test_size = 0.33, random_state = seed)

In [20]:
#pip install tensorflow

In [21]:
# Use scikit-learn to grid search the batch size and epochs

import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Input

X_train.shape

(9681, 6)

In [22]:
y_train.shape

(9681, 1)

In [23]:
from keras import models
from keras import layers
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers
#from keras.layers.normalization import BatchNormalization

In [24]:
in_dim = X_train.shape[1]
out_dim = y_train.shape[1]
in_dim

6

In [25]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

# create model
def model():
        # create model
    model = Sequential()
    model.add(Dense(600, activation='relu', kernel_initializer='normal',input_dim=in_dim))
    model.add(Dense(198, activation='relu', kernel_initializer='normal'))
    model.add(Dense(60, activation='relu', kernel_initializer='normal'))
    model.add(Dense(6, activation='relu', kernel_initializer='normal'))


    # output layer
    model.add(Dense(out_dim))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model


In [26]:
##### create estimator
from keras.wrappers.scikit_learn import KerasRegressor

estimator=KerasRegressor(build_fn=model,epochs=5, batch_size=180,verbose=2)
estimator.fit(X_train,y_train)

C:\Users\dudad\AppData\Local\Temp\ipykernel_14408\914437461.py:4: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator=KerasRegressor(build_fn=model,epochs=5, batch_size=180,verbose=2)


Epoch 1/5
54/54 - 1s - loss: 3833.2371 - 1s/epoch - 19ms/step
Epoch 2/5
54/54 - 0s - loss: 202.6515 - 174ms/epoch - 3ms/step
Epoch 3/5
54/54 - 0s - loss: 160.2452 - 171ms/epoch - 3ms/step
Epoch 4/5
54/54 - 0s - loss: 152.4220 - 190ms/epoch - 4ms/step
Epoch 5/5
54/54 - 0s - loss: 148.2906 - 183ms/epoch - 3ms/step


In [27]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits = 5, shuffle = True)
results = cross_val_score(estimator, X_test, y_test, cv=kfold)
print("Larger: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Epoch 1/5
22/22 - 1s - loss: 9872.3467 - 832ms/epoch - 38ms/step
Epoch 2/5
22/22 - 0s - loss: 1610.0162 - 74ms/epoch - 3ms/step
Epoch 3/5
22/22 - 0s - loss: 387.1301 - 73ms/epoch - 3ms/step
Epoch 4/5
22/22 - 0s - loss: 224.9217 - 75ms/epoch - 3ms/step
Epoch 5/5
22/22 - 0s - loss: 189.7462 - 75ms/epoch - 3ms/step
6/6 - 0s - loss: 173.3479 - 141ms/epoch - 24ms/step
Epoch 1/5
22/22 - 1s - loss: 5736.8213 - 912ms/epoch - 41ms/step
Epoch 2/5
22/22 - 0s - loss: 628.3188 - 70ms/epoch - 3ms/step
Epoch 3/5
22/22 - 0s - loss: 251.7676 - 69ms/epoch - 3ms/step
Epoch 4/5
22/22 - 0s - loss: 190.1722 - 69ms/epoch - 3ms/step
Epoch 5/5
22/22 - 0s - loss: 174.1008 - 68ms/epoch - 3ms/step
6/6 - 0s - loss: 173.0701 - 94ms/epoch - 16ms/step
Epoch 1/5
22/22 - 1s - loss: 7879.1406 - 794ms/epoch - 36ms/step
Epoch 2/5
22/22 - 0s - loss: 824.0070 - 71ms/epoch - 3ms/step
Epoch 3/5
22/22 - 0s - loss: 266.3994 - 69ms/epoch - 3ms/step
Epoch 4/5
22/22 - 0s - loss: 194.4428 - 70ms/epoch - 3ms/step
Epoch 5/5
22/22 - 0

In [28]:
estimator2 = KerasRegressor(build_fn=model,epochs=5, batch_size=180,verbose=2)
estimator2.fit(X_train1,y_train1)

Epoch 1/5


C:\Users\dudad\AppData\Local\Temp\ipykernel_14408\3133084224.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator2 = KerasRegressor(build_fn=model,epochs=5, batch_size=180,verbose=2)


54/54 - 1s - loss: 5649.8682 - 872ms/epoch - 16ms/step
Epoch 2/5
54/54 - 0s - loss: 257.5463 - 162ms/epoch - 3ms/step
Epoch 3/5
54/54 - 0s - loss: 168.7799 - 162ms/epoch - 3ms/step
Epoch 4/5
54/54 - 0s - loss: 158.8256 - 162ms/epoch - 3ms/step
Epoch 5/5
54/54 - 0s - loss: 153.3126 - 160ms/epoch - 3ms/step


In [29]:
from sklearn.model_selection import KFold
kfold1 = KFold(n_splits = 5, shuffle = True)
results1 = cross_val_score(estimator2, X_test1, y_test1, cv=kfold1)
print("Larger: %.2f (%.2f) MSE" % (results1.mean(), results1.std()))

Epoch 1/5
22/22 - 1s - loss: 9418.8369 - 781ms/epoch - 35ms/step
Epoch 2/5
22/22 - 0s - loss: 1573.5967 - 67ms/epoch - 3ms/step
Epoch 3/5
22/22 - 0s - loss: 366.4372 - 67ms/epoch - 3ms/step
Epoch 4/5
22/22 - 0s - loss: 209.2677 - 67ms/epoch - 3ms/step
Epoch 5/5
22/22 - 0s - loss: 178.7523 - 67ms/epoch - 3ms/step
6/6 - 0s - loss: 165.1544 - 98ms/epoch - 16ms/step
Epoch 1/5
22/22 - 1s - loss: 7515.4819 - 782ms/epoch - 36ms/step
Epoch 2/5
22/22 - 0s - loss: 674.7274 - 67ms/epoch - 3ms/step
Epoch 3/5
22/22 - 0s - loss: 262.7299 - 69ms/epoch - 3ms/step
Epoch 4/5
22/22 - 0s - loss: 190.3743 - 70ms/epoch - 3ms/step
Epoch 5/5
22/22 - 0s - loss: 173.6728 - 70ms/epoch - 3ms/step
6/6 - 0s - loss: 170.7621 - 93ms/epoch - 16ms/step
Epoch 1/5
22/22 - 1s - loss: 7552.4180 - 781ms/epoch - 36ms/step
Epoch 2/5
22/22 - 0s - loss: 693.6421 - 68ms/epoch - 3ms/step
Epoch 3/5
22/22 - 0s - loss: 245.2646 - 68ms/epoch - 3ms/step
Epoch 4/5
22/22 - 0s - loss: 190.0036 - 69ms/epoch - 3ms/step
Epoch 5/5
22/22 - 0s

In [30]:
# Compile model
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
#estimator = model

In [32]:
#from keras.optimizers import SGD
#from matplotlib import pyplot

#history = estimator.fit(X_train, y_train, epochs=20, batch_size=1100, validation_data=(X_test, y_test))
# evaluate the model
#_, train_acc = estimator.evaluate(X_train, y_train, verbose=0)
#_, test_acc = estimator.evaluate(X_test, y_test, verbose=0)
#print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot loss during training
#pyplot.subplot(211)
#pyplot.title('Loss')
#pyplot.plot(history.history['loss'], label='train')
##pyplot.plot(history.history['val_loss'], label='test')
#pyplot.legend()
# plot accuracy during training
#pyplot.subplot(212)
##pyplot.title('Accuracy')
#pyplot.plot(history.history['accuracy'], label='train')
#pyplot.plot(history.history['val_accuracy'], label='test')
#pyplot.legend()
#pyplot.show()

# Let's try predicting on the current season


In [33]:
df2 = pd.read_excel('NBA_Game_Data.xlsx')
df2.head()

,Date,Away_Team,Home_Team,Away_stats,Visitor_Team_ID,Home_stats,Home_Team_ID,REB_away,AST_away,PTS_away,...,FG3_PCT_away,FT_PCT_away,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,PTS_home,AWAY_TEAM_W_TODATE,HOME_TEAM_W_TODATE
0,2022-11-01,Golden State Warriors,Miami Heat,Golden State,1.610613e+09,Miami,1.610613e+09,44.6,29.8,118.9,...,0.385,0.794,0.460,0.344,0.831,40.6,23.8,109.5,44.0,44.0
1,2022-11-01,Minnesota Timberwolves,Phoenix Suns,Minnesota,1.610613e+09,Phoenix,1.610613e+09,41.9,26.2,115.8,...,0.365,0.755,0.467,0.374,0.793,44.2,27.3,113.6,42.0,45.0
2,2022-11-01,Orlando Magic,Oklahoma City Thunder,Orlando,1.610613e+09,Oklahoma City,1.610613e+09,43.2,23.2,111.4,...,0.346,0.784,0.465,0.356,0.809,43.6,24.4,117.5,34.0,40.0
3,2022-11-01,Chicago Bulls,Brooklyn Nets,Chicago,1.610613e+09,Brooklyn,1.610613e+09,42.4,24.5,113.1,...,0.361,0.809,0.487,0.378,0.800,40.5,25.5,113.4,40.0,45.0
4,2022-11-02,Memphis Grizzlies,Portland Trail Blazers,Memphis,1.610613e+09,Portland,1.610613e+09,46.6,26.0,116.9,...,0.351,0.733,0.474,0.365,0.796,40.5,24.2,113.4,51.0,33.0


In [34]:
df2.columns

Index(['Date', 'Away_Team', 'Home_Team', 'Away_stats', 'Visitor_Team_ID',
       'Home_stats', 'Home_Team_ID', 'REB_away', 'AST_away', 'PTS_away',
       'FG_PCT_away', 'FG3_PCT_away', 'FT_PCT_away', 'FG_PCT_home',
       'FG3_PCT_home', 'FT_PCT_home', 'REB_home', 'AST_home', 'PTS_home',
       'AWAY_TEAM_W_TODATE', 'HOME_TEAM_W_TODATE'],
      dtype='object')

In [35]:
table = df2[['FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'HOME_TEAM_W_TODATE']]
table1 = df2[['FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away','REB_away','AWAY_TEAM_W_TODATE']]
table.head()

,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,HOME_TEAM_W_TODATE
0,0.460,0.831,0.344,23.8,40.6,44.0
1,0.467,0.793,0.374,27.3,44.2,45.0
2,0.465,0.809,0.356,24.4,43.6,40.0
3,0.487,0.800,0.378,25.5,40.5,45.0
4,0.474,0.796,0.365,24.2,40.5,33.0


In [36]:
y_pred = estimator.predict(table.astype(numpy.float32))
y_pred1 = estimator2.predict(table1.astype(numpy.float32))
y_pred.shape

7/7 - 0s - 108ms/epoch - 15ms/step
7/7 - 0s - 64ms/epoch - 9ms/step


(1186,)

In [37]:
print(y_pred[:5])

[104.8893   116.11263  108.4227   108.687065 103.92986 ]


In [38]:
#y_pred_ = pd.DataFrame(y_pred,columns=['pdw','phw','paw'])
#y_pred_

In [39]:
df2.shape

(1186, 21)

In [40]:
df2['a_pred'] = y_pred1
df2['h_pred'] = y_pred

df2['home_to_win_pred'] = np.where(df2.h_pred > df2.a_pred,1,0)

df2.head()

,Date,Away_Team,Home_Team,Away_stats,Visitor_Team_ID,Home_stats,Home_Team_ID,REB_away,AST_away,PTS_away,...,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,PTS_home,AWAY_TEAM_W_TODATE,HOME_TEAM_W_TODATE,a_pred,h_pred,home_to_win_pred
0,2022-11-01,Golden State Warriors,Miami Heat,Golden State,1.610613e+09,Miami,1.610613e+09,44.6,29.8,118.9,...,0.344,0.831,40.6,23.8,109.5,44.0,44.0,121.654915,104.889297,0
1,2022-11-01,Minnesota Timberwolves,Phoenix Suns,Minnesota,1.610613e+09,Phoenix,1.610613e+09,41.9,26.2,115.8,...,0.374,0.793,44.2,27.3,113.6,42.0,45.0,110.906693,116.112633,1
2,2022-11-01,Orlando Magic,Oklahoma City Thunder,Orlando,1.610613e+09,Oklahoma City,1.610613e+09,43.2,23.2,111.4,...,0.356,0.809,43.6,24.4,117.5,34.0,40.0,105.331703,108.422699,1
3,2022-11-01,Chicago Bulls,Brooklyn Nets,Chicago,1.610613e+09,Brooklyn,1.610613e+09,42.4,24.5,113.1,...,0.378,0.800,40.5,25.5,113.4,40.0,45.0,107.828094,108.687065,1
4,2022-11-02,Memphis Grizzlies,Portland Trail Blazers,Memphis,1.610613e+09,Portland,1.610613e+09,46.6,26.0,116.9,...,0.365,0.796,40.5,24.2,113.4,51.0,33.0,116.939484,103.929863,0


In [41]:
df2.to_excel('nba_preds22.xlsx', index=0)